In [1]:
from iterative_retrieval import IterativeRetrieval
rag = IterativeRetrieval("../context/knowledge_base")

Loading transformer model and tokenizer from transformers library: avsolatorio/NoInstruct-small-Embedding-v0
Please wait...

Loading transformer model and tokenizer from transformers library: meta-llama/Meta-Llama-3.1-8B-Instruct
Please wait...



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# Combine sub-questions into a multi-hop question

In [4]:
from model_prompts import Prompt

In [ ]:
query = "How many years lapsed between Abraham Lincoln's birth and the invention of ASCII?"
contexts = [
    "Abraham Lincoln was born on February 12, 1809."
]

answer = rag.qd.answer_question_using_context(query, contexts, use_chain_of_thought=True)
answer

In [2]:
query = "How many years lapsed between Abraham Lincoln's birth and the invention of ASCII?"
contexts = [
    "Abraham Lincoln was born on February 12, 1809.",
    "Work on the ASCII standard began in May 1961."
]

answer = rag.qd.answer_question_using_context(query, contexts, use_chain_of_thought=True, max_tokens=300)
answer

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
c:\Users\Alex\anaconda3\envs\gpu_tf\lib\site-packages\transformers\models\llama\modeling_llama.py:655: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


"The question is asking for the difference between the year of Abraham Lincoln's birth and the year of invention of ASCII. The context shows that Abraham Lincoln was born in the year 1809, whilst the ASCII standard began in 1961. 196"

In [2]:
query = "How many years lapsed between Abraham Lincoln's birth and the invention of ASCII?"
contexts = [
    "Abraham Lincoln was born on February 12, 1809.",
    "Work on the ASCII standard began in May 1961."
]

is_answer_attainable, chat_history = rag.is_answer_attainable(query, contexts)

print(f"Is answer attainable (should be True): {is_answer_attainable}")
chat_history

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
c:\Users\Alex\anaconda3\envs\gpu_tf\lib\site-packages\transformers\models\llama\modeling_llama.py:655: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Is answer attainable (should be True): False
[{'role': 'system', 'content': '\n    Based on the input and the question, you have to tell me if the information provided is sufficient to answer the question, or whether follow-up questions are needed to find the answer.\n    Your answer must be yes or no.\n    \n    ## Examples:\n    Input: The mother of Antiochus X Eusebes is Cleopatra IV.\n    Input: The father of Cleopatra IV is Ptolemy VIII Physcon.\n    Question: Who is the maternal grandfather of Antiochus X Eusebes?\n    Are follow up questions needed here: No.\n\n    Input: Are both the directors of Jaws and Casino Royale from the same country?\n    Input: The director of Jaws is Steven Spielberg.\n    Question: Steven Spielberg is from the United States.\n    Are follow up questions needed here: Yes.\n\n    Input: Are both the directors of Jaws and Casino Royale from the same country?\n    Input: The director of Jaws is Steven Spielberg.\n    Input: Steven Spielberg is from the U

In [4]:
query = "How many years lapsed between Abraham Lincoln's birth and the invention of ASCII?"
contexts = [
    "Abraham Lincoln was born on February 12, 1809."
]

is_answer_attainable, chat_history = rag.is_answer_attainable(query, contexts)

print(f"Is answer attainable (should be False): {is_answer_attainable}")
chat_history

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Is answer attainable (should be False): False


[{'role': 'system',
  'content': '\n    Based on the input and the question, you have to tell me if the information provided is sufficient to answer the question, or whether follow-up questions are needed to find the answer.\n    Your answer must be yes or no.\n    \n    ## Examples:\n    Input: The mother of Antiochus X Eusebes is Cleopatra IV.\n    Input: The father of Cleopatra IV is Ptolemy VIII Physcon.\n    Question: Who is the maternal grandfather of Antiochus X Eusebes?\n    Are follow up questions needed here: No.\n\n    Input: Are both the directors of Jaws and Casino Royale from the same country?\n    Input: The director of Jaws is Steven Spielberg.\n    Question: Steven Spielberg is from the United States.\n    Are follow up questions needed here: Yes.\n\n    Input: Are both the directors of Jaws and Casino Royale from the same country?\n    Input: The director of Jaws is Steven Spielberg.\n    Input: Steven Spielberg is from the United States.\n    Input: The director of C

In [3]:
query = "How many years lapsed between Abraham Lincoln's birth and the invention of ASCII?"
contexts = [
    "Abraham Lincoln was born on February 12, 1809.",
    "Work on the ASCII standard began in May 1961."
]

is_answer_attainable = rag.is_answer_attainable(query, contexts)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
c:\Users\Alex\anaconda3\envs\gpu_tf\lib\site-packages\transformers\models\llama\modeling_llama.py:655: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [6]:
bool(is_answer_attainable[0])

True

In [11]:
chat_history, sub_question = rag.qd.decompose_question_step(chat_history)

print(sub_question)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


When was the ASCII standard completed and officially published?


In [9]:
a = [{"a"}, {"b"}]

b = [{"c"}, *a]

b

[{'c'}, {'a'}, {'b'}]

In [ ]:
chat_history = rag.answer_multi_hop_question("How many years lapsed between Abraham Lincoln's birth and the invention of ASCII?")

print(chat_history)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Decomposing question: How many years lapsed between Abraham Lincoln's birth and the invention of ASCII?


c:\Users\Alex\anaconda3\envs\gpu_tf\lib\site-packages\transformers\models\llama\modeling_llama.py:655: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Extracted sub-question: To determine the number of years that lapsed between Abraham Lincoln's birth and the invention of ASCII, I'll need to know a few things first. What year was Abraham Lincoln born in?
Attempting to answer sub-question...
Attempt 1 to answer question


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Retrieved context from article: Abraham Lincoln
Attempting to answer question using context: Lincoln was largely self-educated. His formal schooling was from itinerant teachers. It included two short stints in Kentucky, where he learned to read, but probably not to write. In Indiana at age seven, due to farm chores, he attended school only sporadically, for a total of fewer than 12 months in aggregate by age 15. Nonetheless, he remained an avid reader and retained a lifelong interest in learning. Family, neighbors, and schoolmates recalled that his readings included the King James Bible, Aesop's Fables, John Bunyan's "The Pilgrim's Progress", Daniel Defoe's "Robinson Crusoe", and "The Autobiography of Benjamin Franklin". Despite being self-educated, Lincoln was the recipient of honorary degrees later in life, including an honorary Doctor of Laws from Columbia University in June 1861.


In [4]:
final_answer = rag.qd.answer_question_using_context(
    "How many years lapsed between Abraham Lincoln's birth and the invention of ASCII?",
    " ".join([
        "Abraham Lincoln was born on February 12, 1809.",
        "Work on the ASCII standard began in May 1961."
    ])
)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [5]:
final_answer

"There are 52 years between Abraham Lincoln's birth and the invention of ASCII."

In [ ]:
chat_history = [
    {'role':'system','content':}
]